In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv")
train_df.shape

In [ ]:
train_df.head(20) # each row corresponds to combination of {date,country,store,product} and {the numbers of that product sold}

In [ ]:
train_df.tail(2)

In [ ]:
train_df = train_df.drop("row_id", axis=1)

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv")
test_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df=test_df.drop("row_id",axis=1)

In [ ]:
train_df.dtypes

In [ ]:
train_df.columns

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

#### Max items sold is 2884. Average products sold is 388

In [ ]:
train_df.describe(include=np.object)

#### 3 country, 2 store and 3 products in the training data

In [ ]:
train_df.store.unique()

In [ ]:
train_df.store.value_counts()

In [ ]:
train_df.country.unique()

In [ ]:
train_df.country.value_counts()

In [ ]:
train_df['product'].unique()

In [ ]:
train_df['product'].value_counts()

In [ ]:
sum(train_df.num_sold==0)

In [ ]:
sum(train_df.num_sold <= 0)

#### 3 products Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker'
#### country Sweden, Finland and Norway
#### Store KaggleMart and KaggleRama

In [ ]:
train_df.loc[:,'date'] = pd.to_datetime(train_df.date)
train_df.dtypes

In [ ]:
train_df['Year'] = train_df.date.dt.year

In [ ]:
train_df['Month'] = train_df.date.dt.month

In [ ]:
train_df.head()

In [ ]:
 train_df_Sweden = train_df[train_df.country == "Sweden"]
 train_df_Finland = train_df[train_df.country == "Finland"]
 train_df_Norway = train_df[train_df.country == "Norway"]

In [ ]:
train_df_Sweden

#### The line plot below shows an upward trend for both the store from the year 2015 to year 2018. The products sold higher for KaggleRama then KaggleMart

In [ ]:
sns.set(rc={'figure.figsize': (15, 5)}, style='white')
ax = sns.lineplot(data=train_df_Sweden, x='Year', y='num_sold', hue='store')
ax.set_title("Products sold in Sweden")
plt.show()

#### For Finland below the Kaggle Hat sold more. We can see there is Trend, Seasonality and Noise in the data below.

In [ ]:
sns.set(rc={'figure.figsize': (20, 10)},style='white', font_scale=2)
g = sns.FacetGrid(train_df_Finland, col='product', height=10)
g = g.map(plt.plot, 'date', 'num_sold')
g.set_titles(size=25)
g.set_xticklabels(rotation=45)
plt.show()

#### KaggleRama doing well

In [ ]:
sns.set(rc={'figure.figsize': (15, 5)}, style='white')
ax = sns.lineplot(data=train_df_Finland, x='Year', y='num_sold', hue='store')
ax.set_title("Products sold in Sweden")
plt.show()

#### Kaggle Hat product is doing good.

In [ ]:
sns.set(rc={'figure.figsize': (18, 5)}, style='white')
ax = sns.lineplot(data=train_df_Sweden, x='Year', y='num_sold', hue='product')
ax.set_title("Products sold in Sweden")
plt.show()

In [ ]:
train_df_Norway2018 = train_df_Norway[train_df_Norway['Year']==2018]
train_df_Norway2017 = train_df_Norway[train_df_Norway['Year']==2017]

In [ ]:
sns.set(rc={'figure.figsize': (15, 5)}, style='white')
ax = sns.lineplot(data=train_df_Norway2018, x='Month', y='num_sold',style='store',hue='store')
ax.set_title("Products sold in Norway for the year 2018")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize': (15, 5)}, style='white')
ax = sns.lineplot(data=train_df_Norway2017, x='Month', y='num_sold',style='store',hue='store')
ax.set_title("Products sold in Norway for the year 2017")
plt.show()

In [ ]:
train_df[(train_df.store == 'KaggleRama') & (train_df.country=="Sweden")].nlargest(n=10, columns='num_sold') # highest sale in december 

In [ ]:
train_df[(train_df.store == 'KaggleMart') & (train_df.country=="Sweden")].nlargest(n=10, columns='num_sold') # december doing good in sale across all the years

In [ ]:
train_df[(train_df.store == 'KaggleRama') & (train_df.country=="Finland")].nlargest(n=10, columns='num_sold')

In [ ]:
train_df[(train_df.store == 'KaggleMart') & (train_df.country=="Finland")].nlargest(n=10, columns='num_sold')

#### Same for Finland

In [ ]:
train_df[(train_df.store == 'KaggleRama') & (train_df.country=="Norway")].nlargest(n=10, columns='num_sold')

In [ ]:
train_df[(train_df.store == 'KaggleMart') & (train_df.country=="Norway")].nlargest(n=10, columns='num_sold')

#### For Norway same except one in April 2018.

In [ ]:
 train_df[train_df.duplicated()].shape[0]

#### Rolling 7 day mean

In [ ]:
new_df = train_df.pivot_table(
index='date',
columns='country',
values='num_sold'
).rolling(7).mean()

In [ ]:
new_df ## average sale for window of 7 days we can see Norway doing better among the three countries

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
new_df[['Finland']].plot(ax=axes[0], style='-.c')
new_df[['Sweden']].plot(ax=axes[1], style=['-', '--'], title='7-day rolling average of new ')
new_df[['Norway']].plot(ax=axes[2], style=['--', ':', '-'])

In [ ]:
new_df = train_df.pivot_table(
index='date',
columns='product',
values='num_sold'
).rolling(7).mean()

In [ ]:
new_df ## average sale for window of 7 days we can see product Kaggle Hat doing better among the three products

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
new_df[['Kaggle Hat']].plot(ax=axes[0], style='-.c')
new_df[['Kaggle Mug']].plot(ax=axes[1], style=['-', '--'], title='7-day rolling average of new ')
new_df[['Kaggle Sticker']].plot(ax=axes[2], style=['--', ':', '-'])

In [ ]:
new_df = train_df.pivot_table(
index='date',
columns='store',
values='num_sold'
).rolling(7).mean()

In [ ]:
new_df #### KaggleRama doing better in the sales

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
new_df[['KaggleMart']].plot(ax=axes[0], style='-.c')
new_df[['KaggleRama']].plot(ax=axes[1], style=['-', '--'], title='7-day rolling average of new ')

In [ ]:
train_df.num_sold.plot(kind='hist')

In [ ]:
train_df[['num_sold',"country"]].groupby('country').boxplot(figsize=(15,8), subplots=False)

In [ ]:
train_df[['num_sold',"store"]].groupby('store').boxplot(figsize=(15,8), subplots=False)

In [ ]:
train_df[['num_sold',"product"]].groupby('product').boxplot(figsize=(15,8), subplots=False)

In [ ]:
sns.stripplot(
x='product', 
y='num_sold', 
hue='store',
data=train_df
)

In [ ]:
sns.stripplot(
x='country', 
y='num_sold', 
hue='store',
data=train_df
)

In [ ]:
sns.stripplot(
x='country', 
y='num_sold', 
hue='product',
data=train_df
)

#### From the above plots we can see Norway doing good in terms of products sale. KaggleRama store is doing good. The product Kaggle Hat is doing good.

In [ ]:
train_df.groupby('country').agg({'num_sold': ['mean', 'min', 'max']})

#### Norway --> Sweden --> Finland in sale

In [ ]:
train_df.groupby('Year').agg({'num_sold': ['mean', 'min', 'max']})

#### The sale increased from 2015 --> 2018. average sale same for the year 2015 and 2016 but then increased to 2017 and 2018

In [ ]:
train_df.groupby('store').agg({'num_sold': ['mean', 'min', 'max']})

#### KaggleRama --> KaggleMart

In [ ]:
train_df.groupby('product').agg({'num_sold': ['mean', 'min', 'max']})

#### Kaggle Hat highest in sale

In [ ]:
train_df.groupby(['Year','country']).agg({'num_sold': ['mean', 'min', 'max']})

#### Across the year Norway has done good

In [ ]:
train_df.groupby(['Year','country','store']).agg({'num_sold': ['mean', 'min', 'max']})

#### 2018 --> Norway --> KaggleRama
#### 2017 --> Norway --> KaggleRama

In [ ]:
train_df.groupby(['Year','country','store','product']).agg({'num_sold': ['mean', 'min', 'max']}).head(36)

In [ ]:
train_df.groupby(['Year','country','store','product']).agg({'num_sold': ['mean', 'min', 'max']}).tail(36)

#### 2018 --> Norway --> KaggleRama --> Kaggle Hat.

#### 2017 --> Norway --> KaggleRama --> Kaggle Hat.

In [ ]:
sns.swarmplot(
x='product', 
y='num_sold', 
hue='store',
data=train_df_Norway2018
)

In [ ]:
sns.swarmplot(
x='product', 
y='num_sold', 
hue='store',
data=train_df_Norway2017
)

In [ ]:
sns.swarmplot(
x='product', 
y='num_sold', 
hue='store',
data=train_df_Sweden
)

#### The above three plots, we can say that KaggleRama store and the number of Kaggle Hat sold is high